In [5]:
from selenium import * webdriver 
from time import sleep
from PIL import Image
from io import BytesIO
from tqdm import tqdm

In [102]:
initLink = "https://yandex.ru/maps/213/moscow/?ll=37.620853%2C55.752200&mode=whatshere&whatshere%5Bpoint%5D=37.616644%2C55.750826&whatshere%5Bzoom%5D=15.44&z=15.44"

width = 3000
height = 5000

boundSize = 500
boundSizeHalf = boundSize//2

sidebar = {
    "yandex": "sidebar-toggle-button__icon"
}

In [104]:
def slide (browser, x,y):
    action = webdriver.ActionChains(browser)
    action.move_to_element(mapElement).perform()    
    action.click_and_hold(mapElement).move_by_offset(x,y).perform()
    sleep(1) # change sleepTime for your internet connection
    action.release().perform() 
    sleep(1)          

In [ ]:
# you should set path to chromedriver https://chromedriver.chromium.org/downloads

browser = webdriver.Chrome()

In [103]:
browser.set_window_size(2000,2000)
browser.get(initLink)

distanceToStartX = width //2 - boundSizeHalf
distanceToStartY = height //2 - boundSizeHalf 

sidebar = next((x[1] for x in sidebar.items() if x[0] in initLink), None)

browser.find_element_by_class_name(sidebar).click()

In [105]:
mapElement = browser.find_element_by_tag_name("body")

In [106]:
selectShift = lambda x: x if x < boundSize else boundSize
while 1:
    toMoveX = selectShift(distanceToStartX)
    toMoveY = selectShift(distanceToStartY)
#     mapElement = browser.find_element_by_tag_name("body")
    slide(browser, toMoveX,toMoveY)
    distanceToStartX -=toMoveX
    distanceToStartY -=toMoveY
    if distanceToStartX == 0 and distanceToStartY==0:
        break
    print(toMoveX, toMoveY, distanceToStartX, distanceToStartY)

500 500 750 1750
500 500 250 1250
250 500 0 750
0 500 0 250


In [107]:
smallScreenShoot =lambda im: im.crop([x//2 + s for s in [-boundSizeHalf,boundSizeHalf] for x in im.size ] )
finalScreenShot = Image.new('RGB', (width, height))

sign = 1
for y in  range(0, height,boundSize ):
    for x in  range(0, width,boundSize ):
        png = browser.get_screenshot_as_png() 
        im = Image.open(BytesIO(png))
        finalScreenShot.paste(smallScreenShoot(im), ( x,y))
        slide(browser, -boundSize * sign,0)        
    slide(browser, 0,-boundSize)        
    sign *=-1
        
finalScreenShot.save("resultmap.png")              